In [1]:
import spacy
import pandas as pd
import numpy as np

import Modules.solrhandler as sh
import Modules.clusterer as cls
import Modules.topicdeterminator as td
import Modules.chatbot as cb

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
#query = "anmeldung"
#target_service = 120686

In [3]:
# Import eval.csv
import os
file = os.path.realpath("../../Evaluierung/eval.csv")
eval = pd.read_csv(file, sep = ";")
eval = eval.head(500)
eval.head(20)

,documentId,file,dialogId,initialQuestion,suggestionCount,directlyFound,name
0,120697,20220131--quantEvalAllExcel.xlsx,4,Ummeldung,6,True,Änderung/Wechsel der Hauptwohnung
1,120686,20220131--quantEvalAllExcel.xlsx,4,Ummelden,6,True,Anmeldung einer Wohnung
2,329421,20220131--quantEvalAllExcel.xlsx,19,Pcr test positiv brauche ich eine Krankschreib...,6,False,Erstattung nach Infektionsschutzgesetz bei Tät...
3,329421,20220131--quantEvalAllExcel.xlsx,19,Quarantäne Krankschreibung vom Arzt,6,True,Erstattung nach Infektionsschutzgesetz bei Tät...
4,120914,20220131--quantEvalAllExcel.xlsx,23,Termin für Zulassung Gebrauchtwagen,6,True,Zulassung eines Fahrzeuges mit auswärtigem Ken...
5,328200,20220131--quantEvalAllExcel.xlsx,24,Wo kann ich mich nach Corona Infektion freites...,6,True,Infektionskrankheiten melden - für Gemeinschaf...
6,330073,20220131--quantEvalAllExcel.xlsx,28,Ab wann ist man bei Johnson & Johnson geboostert,5,True,Impfung gegen Corona (SARS-CoV-2)
7,120702,20220131--quantEvalAllExcel.xlsx,36,Genesenen Bescheinigung,6,True,Meldebescheinigung beantragen
8,330150,20220131--quantEvalAllExcel.xlsx,39,"welche medikamente kann man nehmen, wenn man c...",6,True,Arzneimittel - zur Sachkundeprüfung für den Ha...
9,330174,20220131--quantEvalAllExcel.xlsx,47,ab 15.1. ist für Gäste von Restaurants Nachwe...,6,True,Gaststättengewerbe - zum Unterrichtungsnachwei...


In [4]:
#eval.iloc[2]["initialQuestion"]

Funktionen

In [5]:
def findCorrectAnswer(chatbot,targetService):

    # Find Target Cluster
    clusteredColumn = chatbot.clusterer.getClusteredColumn()
    service = chatbot.df.loc[chatbot.df["id"]==str(targetService)]
    if len(service[clusteredColumn].values) == 0:
        return None

    targetCluster = service[clusteredColumn].values[0]

    if targetCluster == chatbot.getSelectedClusterForQuestion():
        return True

    else:
        return False

def getRank(chatbot, targetService):
    temp = chatbot.df.reset_index()
    try:
        rank = temp.index[temp["id"]==str(targetService)].values[0]
    except IndexError as e:
        return float("NaN")
    return rank + 1

Logging

In [6]:
class Logger:
    
    def __init__(self, eval_df):
        self.liste = []
        self.eval_df = eval_df
        self.chatbot = None
    
    def getDataFrame(self): # TODO
        pass
    
    def setChatbot(self, chatbot):
        self.chatbot = chatbot
    
    def log(self, ID, t, row, answer = None, initialQuery = False): 
        target_service = row["documentId"]
        
        dialogId = row["dialogId"]
        file = row["file"]
        
        service_name = row["name"]
        query = row["initialQuestion"]
        
        if initialQuery:
            nResults = len(self.chatbot.df.index)
            rank = getRank(self.chatbot,target_service)# getRank(target_service)

            row = (file, dialogId, ID, t, service_name, query, "initialQuery", None, rank, nResults)
        else:
            if answer is not None:


                nResults = len(self.chatbot.df.index)
                question = self.chatbot.generateQuestion()
                answer = answer

                rank = getRank(self.chatbot,target_service)# getRank(target_service)

                row = (file, dialogId, ID, t,  service_name, query, question, answer, rank, nResults)
            else:
                row = (file, dialogId, ID, t, service_name, query, None, None, None, None)
        self.liste.append(row)
        #print(self.liste)

TEST

Execution

In [7]:
# Initialisierung Komponenten Chatbot
#solrhandler = sh.SolrHandler(1000)
#clusterer = cls.Clusterer()
#tpc_dterminator = td.TopicDeterminator()

from sklearn.cluster import DBSCAN


In [8]:
# Experiment durchlaufen und Loggen
lg = Logger(eval_df = eval)   

for (ID, row) in eval.iterrows():
    #print(ID)
    
    query = row["initialQuestion"]
    target_service = row["documentId"]
    t = 0
    
    # Chatbot initialisieren
    try: # 1.3
        chatbot = cb.Chatbot(query,3, forceClusters = True)
    
    # Fehler überspringen: eg. kein solr output
    except Exception as e: 
        if(str(e) == 'response'): # TODO ??
            next
        elif(str(e) == 'no solr output'):
            next
        else:
            print(e)
            
    lg.setChatbot(chatbot)

    
    answer = findCorrectAnswer(chatbot, target_service)
    
    if answer is None: # service nicht in resultset, überspringen
        next
    lg.log(ID,t, row, answer, initialQuery = True)
    
    while not chatbot.isFinished():#chatbot.refineResultset(answer, recluster = True):

        # log
        t += 1
        
#         print(ID)
#         print(t)
#         print(np.unique(chatbot.df["ssdsLemma_cluster"]))
#         print("")
        
        chatbot.refineResultset(answer, True)
        answer = findCorrectAnswer(chatbot, target_service)
        if answer is None: 
            break
        lg.log(ID, t, row,answer)
        
        #print(chatbot.clusterer.clustering_algorithm.eps)

1.1700000000000002
1.0530000000000002
'response'
'response'
'response'
'response'
0.9477000000000002
'response'
'response'


In [9]:
log_df = pd.DataFrame(lg.liste, columns=["file","dialogId", "ID","t", "name", "query", "question", "answer", "rank", "nResult"])
log_df[log_df["question"] == "initialQuery"].head(40)

,file,dialogId,ID,t,name,query,question,answer,rank,nResult
0,20220131--quantEvalAllExcel.xlsx,4,0,0,Änderung/Wechsel der Hauptwohnung,Ummeldung,initialQuery,None,3.0,7
3,20220131--quantEvalAllExcel.xlsx,4,1,0,Anmeldung einer Wohnung,Ummelden,initialQuery,None,NaN,7
4,20220131--quantEvalAllExcel.xlsx,19,2,0,Erstattung nach Infektionsschutzgesetz bei Tät...,Pcr test positiv brauche ich eine Krankschreib...,initialQuery,None,4.0,13
7,20220131--quantEvalAllExcel.xlsx,19,3,0,Erstattung nach Infektionsschutzgesetz bei Tät...,Quarantäne Krankschreibung vom Arzt,initialQuery,None,1.0,10
10,20220131--quantEvalAllExcel.xlsx,23,4,0,Zulassung eines Fahrzeuges mit auswärtigem Ken...,Termin für Zulassung Gebrauchtwagen,initialQuery,None,7.0,104
15,20220131--quantEvalAllExcel.xlsx,24,5,0,Infektionskrankheiten melden - für Gemeinschaf...,Wo kann ich mich nach Corona Infektion freites...,initialQuery,None,NaN,1
16,20220131--quantEvalAllExcel.xlsx,28,6,0,Impfung gegen Corona (SARS-CoV-2),Ab wann ist man bei Johnson & Johnson geboostert,initialQuery,None,NaN,2
17,20220131--quantEvalAllExcel.xlsx,36,7,0,Meldebescheinigung beantragen,Genesenen Bescheinigung,initialQuery,None,1.0,58
26,20220131--quantEvalAllExcel.xlsx,39,8,0,Arzneimittel - zur Sachkundeprüfung für den Ha...,"welche medikamente kann man nehmen, wenn man c...",initialQuery,None,NaN,1
27,20220131--quantEvalAllExcel.xlsx,47,9,0,Gaststättengewerbe - zum Unterrichtungsnachwei...,ab 15.1. ist für Gäste von Restaurants Nachwe...,initialQuery,None,13.0,61


Log Analyzer

In [10]:
import Modules.log_analyzer as lo_an

In [11]:
log_analyzer=lo_an.Log_Analyzer(log_df)
delta,services,questions,means=log_analyzer.run_it()

ValueError: too many values to unpack (expected 4)

In [ ]:
log_df.head(40)

In [ ]:
means

Statistics ( Alt)

In [ ]:
# avg Rank Improvement in turn t
log_df["delta_Rank"] = log_df.groupby(["ID"])["rank"].diff()
#log_df.groupby(["t"])["rank_improvement"].mean()

In [ ]:
# avg nSuggestions Improvement in turn t
log_df["delta_nResults"] = log_df.groupby(["ID"])["nResults"].diff()
#log_df.groupby(["t"])["sugg_improvement"].mean()

In [ ]:
# MRR ohne cluster refinement
#df = log_df.loc[log_df["t"] == 1]
#MRR_ohne = 1/(df["rank"].mean())
#MRR_ohne

In [ ]:
# MRR mit cluster refinement
#df = log_df.groupby(["ID"]).last("t")
#MRR_mit = 1/(df["rank"].mean())
#MRR_mit
#df.head()

In [ ]:
# avg Turns
print(log_df["t"].mean())

# bei durchschnittlich nResults
print(log_df.groupby(["ID"]).last("t")["nResults"].mean())

In [ ]:
log_df.head(20)